In [ ]:
!pip install boto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.4 MB/s eta 0:00:00


In [ ]:
import boto
ec2 = boto.connect_ec2(aws_access_key_id='xxx', aws_secret_access_key='xxx')

In [ ]:
# use latest view boto3
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
# question1 : set up instance
!pip install awscli
import boto3
ec2  = boto3.resource(
'ec2',
aws_access_key_id='xxx',
aws_secret_access_key='xxx',
region_name='us-west-2'
)

zsh:1: command not found: pip


ModuleNotFoundError: No module named 'boto3'

In [ ]:
!aws configure --profile test
#!aws ec2 create-key-pair --key-name 'KeyPair1' --query 'KeyMaterial' --output text --region us-west-2 > KeyPair1.pem 
#ec2.create_instances( ImageId="ami-0dafa01c8100180f8",
#        MinCount=1,
#        MaxCount=1,
#        InstanceType="t2.micro",
#        KeyName="keyValuePair"
#    )

AWS Access Key ID [None]: xxx
AWS Secret Access Key [None]: xxx
Default region name [None]: us-west-2
Default output format [None]: 


In [ ]:

!aws ec2 create-key-pair --key-name 'KeyPair1' --query 'KeyMaterial' --output text --profile test > KeyPair1.pem 


In [ ]:
instances = ec2.create_instances( ImageId="ami-095413544ce52437d",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="KeyPair1"
    )

In [ ]:
# question2: add tag for instance
instance = instances[0]
instance.create_tags(DryRun=False, Resources=[instance.id],
                     Tags=[{'Key': 'key1', 'Value': 'value1'}])

[ec2.Tag(resource_id='i-09144a1192356982b', key='key1', value='value1')]

In [ ]:
# question3: reboot the instance
instance.reboot()

{'ResponseMetadata': {'RequestId': '9f4fe798-2e36-4100-9a9f-731523da6cbb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9f4fe798-2e36-4100-9a9f-731523da6cbb',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '231',
   'date': 'Thu, 02 Feb 2023 07:51:26 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [ ]:
# question4: stop the instance
instance.stop()

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'},
   'InstanceId': 'i-09144a1192356982b',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': 'f625f60e-0f38-4bc5-8d43-2a58a1b7618d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f625f60e-0f38-4bc5-8d43-2a58a1b7618d',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '579',
   'date': 'Thu, 02 Feb 2023 07:53:53 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [ ]:
# quetion5 : start 6 instance
instances2 = ec2.create_instances( ImageId="ami-095413544ce52437d",
        MinCount=5,
        MaxCount=6,
        InstanceType="t2.micro",
        KeyName="KeyPair1"
    )
print(len(instances2))

6


In [ ]:
# question 6: assign tag,check if the instance id same( I check all the running instance, can't find the same Id)
for i in range(6):
  suffix = '1' if i % 2 == 0 else '2'
  instances2[i].create_tags(DryRun=False, Resources=[instances2[i].id],
                     Tags=[{'Key': 'key' + suffix, 'Value': 'value' + suffix}])


In [ ]:
# question 7: filter by tag
value1_instances = [ i for i in instances2 if i.tags[0]['Value'] == 'value1']
# set value1 instances' tag to value2
for instance in value1_instances:
  instance.delete_tags(DryRun=False, Resources=[instance.id],
                     Tags=[{'Key': 'key1', 'Value': 'value1'}])
  
for instance in value1_instances:
  instance.create_tags(DryRun=False, Resources=[instance.id],
                     Tags=[{'Key': 'key2', 'Value': 'value2'}])

In [ ]:
# question 8: describe all the instances
# question 9 and 10: all the instance tag update by one tag and list them
for instance in instances2:
  print(str(instance.id) + ',' + str(instance.tags))

i-0b896d91638c3265b,[{'Key': 'key2', 'Value': 'value2'}]
i-0a2c90b815950c82b,[{'Key': 'key2', 'Value': 'value2'}]
i-02d4d308410029709,[{'Key': 'key2', 'Value': 'value2'}]
i-0962fe0ef0afe8687,[{'Key': 'key2', 'Value': 'value2'}]
i-00958664eacd815c4,[{'Key': 'key2', 'Value': 'value2'}]
i-0d3f0e1f6db3ea800,[{'Key': 'key2', 'Value': 'value2'}]


In [ ]:
# question 11: stop the instance
for instance in instances2:
  instance.stop()